In [42]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import norm
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.optimize import curve_fit
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt;
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error as mse
import itertools
plt.rcdefaults()

In [43]:
df_uah1 = pd.read_excel("/Users/Marta/Desktop/diploma/Data_2_S.xlsx", "uah1")
df_uah2 = pd.read_excel("/Users/Marta/Desktop/diploma/Data_2_S.xlsx", "uah2")

In [44]:
df_uah1.set_index("Period", inplace=True)
df_uah1 = df_uah1[["T1", "T2","Total 1"]]
df_uah1.columns = ["T1_df_uah1", "T2_df_uah1","Total 1_df_uah1"]

df_uah2.set_index("Period", inplace=True)
df_uah2 = df_uah2[["T1", "T2","Total 1"]]
df_uah2.columns = ["T1_df_uah2", "T2_df_uah2","Total 1_df_uah2"]

df_all = pd.concat([df_uah1, df_uah2], axis=1)

In [45]:
df = df_uah2
n = "df_uah2"
model_type = "multiplicative"
period = 12

In [186]:
residuals_df = pd.DataFrame(index=df.index, columns=df.columns)
seasonalities_df = pd.DataFrame(index=df.index, columns=df.columns)
trend_df = pd.DataFrame(index=df.index, columns=df.columns)
for col in df.columns:
    additive_decomposition = seasonal_decompose(x=df[col], model=model_type, period=period)
    residuals_df[col] = additive_decomposition.resid
    seasonalities_df[col] = additive_decomposition.seasonal
    trend_df[col] = additive_decomposition.trend

In [3]:
for mon in range(1, 13):
    digit = f"0{mon}" if mon < 10 else f"{mon}"
    seasonalities_df.loc[pd.datetime(2022, mon, 1), :] = seasonalities_df.loc[pd.datetime(2021, mon, 1), :]

for mon in range(1, 13):
    digit = f"0{mon}" if mon < 10 else f"{mon}"
    seasonalities_df.loc[pd.datetime(2023, mon, 1), :] = seasonalities_df.loc[pd.datetime(2021, mon, 1), :]
    
for mon in range(1, 7):
    digit = f"0{mon}" if mon < 10 else f"{mon}"
    seasonalities_df.loc[pd.datetime(2024, mon, 1), :] = seasonalities_df.loc[pd.datetime(2021, mon, 1), :]

In [189]:
trend_df = trend_df.iloc[:33].fillna(0)
residuals_df = residuals_df.iloc[:33].fillna(0)
predicted_residuals = seasonalities_df.copy(deep=True)

In [1]:
s1 = set((0,1, 2, 3, 6, 12))
triples = list(itertools.product(s1,s1,s1))

def predict_arima(X, months=24, colname="y"):
    mses_all = []
    preds_ = []
    mse_ = np.inf
    for i in triples:
        print("i=", i)
        try:
            sar = SARIMAX(X, order=i).fit()

            preds = sar.predict(start = 0, end= months, dynamic=False)
            c = n + "_" + str(colname) + ",SARIMAX("+str(i)+f"), residuals({model_type},{period})"
            print("len_preds", len(preds.fillna(0)))
        except Exception as er:
            print("error", er)
            continue
        
        ar_mse = mse(X, preds[:len(X)].fillna(0))
        mses_all.append((c, ar_mse, preds))
        print("mse = ", ar_mse)
        print("preds = ", len(preds))
        print("preds_ = ", len(preds_))
        if ar_mse < mse_:
            preds_ = preds.fillna(0)
            print("neew preds: ", preds)
            mse_ = ar_mse
    
    return mses_all

for col in residuals_df.columns:
    mses_all_ = predict_arima(residuals_df[col], 66, col)
    mses_all_.sort(key=lambda x:x[1])
    for m in mses_all_[:5]:
        predicted_residuals[m[0]] = m[2]
    
    


In [191]:
predicted_residuals = predicted_residuals[[i for i in predicted_residuals.columns if len(i) > 8]]

In [193]:
# functions to predict trend with

def polynomial4_func(x, a, b, c, d, e):
    return a + b * x + c * (x ** 2) + d * (x ** 3) + e * (x ** 4)
def polynomial5_func(x, a, b, c, d, e, f):
    return (a * x) + (b * x**2) + (c * x**3) + (d * x**4) + (e * x**5) + f
def arctan_func(x, a, b, c, d, e, f, g):
    return - a*np.arctan(x) -b*x**(1/5) -c*x**(1/6) -d*x -e*x**2 -f*x**3 -g*x**4
def hyperbola_func(x, a, b, c):
    return a/x + b/(x**x) +c
def log_func(x, a, b, c):
    return -np.log(a*x) + b/(x**x) + c
def log2_func(x, a, b, c, d):
    return -d*np.log(a*x) + b/(x**x) + c
def log3_func(x, a, b, c, d):
    return d*np.log(a*x) + b/(x**2) + c
def log4_func(x, a, b, c, d, e):
    return d*np.log(a*x) + b*(x**2) + c + e*(x**3)

def predict_trend(y, months=12, func=polynomial4_func):
    x = np.array(range(1, len(y)+1)).astype('float64')
    x_range = np.linspace(1, len(y)+months+1, num=len(y)+months)
    params = curve_fit(func, x, y)
    return func(x_range, *params[0])

funcs = [polynomial4_func, polynomial5_func, arctan_func, hyperbola_func, log_func, log2_func, log3_func]

In [194]:
predicted_trend_df = trend_df.fillna(0).copy(deep=True)
trend_df = trend_df.dropna()
predicted_trend_df

,T1,T2,Total 1
Period,,,
2019-01-01,0.000000e+00,0.000000e+00,0.000000e+00
2019-02-01,0.000000e+00,0.000000e+00,0.000000e+00
2019-03-01,0.000000e+00,0.000000e+00,0.000000e+00
2019-04-01,0.000000e+00,0.000000e+00,0.000000e+00
2019-05-01,0.000000e+00,0.000000e+00,0.000000e+00
2019-06-01,0.000000e+00,0.000000e+00,0.000000e+00
2019-07-01,3.522366e+07,2.550993e+06,2.017332e+08
2019-08-01,3.655827e+07,2.575075e+06,2.058512e+08
2019-09-01,3.797285e+07,2.610656e+06,2.146440e+08


In [2]:
for mon in range(10, 13):
    digit = f"0{mon}" if mon < 10 else f"{mon}"
    predicted_trend_df.loc[pd.datetime(2021, mon, 1), :] = predicted_trend_df.loc[pd.datetime(2020, mon, 1), :]

for mon in range(1, 13):
    digit = f"0{mon}" if mon < 10 else f"{mon}"
    predicted_trend_df.loc[pd.datetime(2022, mon, 1), :] = predicted_trend_df.loc[pd.datetime(2020, mon, 1), :]
    
for mon in range(1, 13):
    digit = f"0{mon}" if mon < 10 else f"{mon}"
    predicted_trend_df.loc[pd.datetime(2023, mon, 1), :] = predicted_trend_df.loc[pd.datetime(2021, mon, 1), :]

for mon in range(1, 7):
    digit = f"0{mon}" if mon < 10 else f"{mon}"
    predicted_trend_df.loc[pd.datetime(2023, mon, 1), :] = predicted_trend_df.loc[pd.datetime(2021, mon, 1), :]

print(predicted_trend_df)
for col in predicted_trend_df.columns:
    min_mse = np.inf
    min_preds = []
    for func in funcs:
        try:
            pred_sequence = predict_trend(trend_df[col], 27, func)
            mse_ = mse(trend_df[col], pred_sequence[:len(trend_df[col])])
            c = n + "_" + str(col) + f",{n},trend({model_type},{period}+{func})"
            print(pred_sequence)
            predicted_trend_df.loc[:,c] = pred_sequence

        except Exception as ex:
            print(ex)
            continue
   

In [196]:
predicted_trend_df = predicted_trend_df[[i for i in predicted_trend_df.columns if len(i) > 8]]

In [177]:
import os
import pandas as pd
# assign directory
directory = '/Users/Marta/Desktop/diploma/best2/'
 
# iterate over files in
# that directory
residuals = pd.DataFrame()
seasonalities = pd.DataFrame()
fulls = pd.DataFrame()
trends = pd.DataFrame()
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        if "RESIDUALS" in f and "uah1" in f and "T1" in f:
#             print(f)
            df = pd.read_csv(f).set_index("Period")
            residuals = pd.concat([residuals, df], axis=1)
        if "SEASONALIT" in f and "uah1" in f and "T1" in f:
#             print(f)
            df = pd.read_csv(f).set_index("Period")
            df.columns = [filename+col for col in df.columns]
            seasonalities = pd.concat([seasonalities, df], axis=1)
        if "FULL" in f and "uah1" in f and "T1" in f:
#             print(f)
            df = pd.read_csv(f).set_index("Period")
            fulls = pd.concat([fulls, df], axis=1)
        if "TREND" in f and "uah1" in f and "T1" in f:
#             print(f)
            df = pd.read_csv(f).set_index("Period")
            trends = pd.concat([trends, df], axis=1)
            
trends_predicted = pd.read_csv("/Users/Marta/Desktop/diploma/trends_predicted.csv")
trends_predicted_regressed = pd.read_csv("/Users/Marta/Desktop/diploma/trends_predicted_regressed.csv")
trends_predicted = pd.concat([trends_predicted, trends_predicted_regressed], axis=1).set_index("Unnamed: 0")
fulls.columns = ["full_"+c for c in fulls.columns]